spotrac 迴歸分析

In [2]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import statsmodels.api as sm #OLS
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

from linearmodels.panel import PanelOLS # FE
# https://bashtage.github.io/linearmodels/doc/panel/models.html

# 讀檔

In [139]:
#野手
df_bat = pd.read_excel(r"G:\我的雲端硬碟\python\output\0828\Spotrac_bat_data_20210828_19.48.49.xlsx",
                        engine='openpyxl')
# 投手
df_pitch= pd.read_excel(r"G:\我的雲端硬碟\python\output\0828\Spotrac_pitch_data_20210828_19.48.58.xlsx",
                      engine='openpyxl')

# 特徵轉換

In [140]:
for df in [df_pitch, df_bat]:
    
    #T,F transform
    for col in [col for col in df_bat.columns if "is_" in col]:
        df.loc[:,col] = df.loc[:,col].apply(lambda v: 1 if v else 0)
        
    # AAV_dollar 取 自然對數
    df.loc[:,"AAV_dollar"] = np.log(df.loc[:,"AAV_dollar"])
    df.loc[np.isinf(df_bat.loc[:,"AAV_dollar"]),"AAV_dollar"] = np.nan

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [19]:
# df_pitch.info()

In [10]:
# df_bat.info()

# Model 1

 WAR = 合約剩餘年數 + 一年合約dummy + 控制變數

## OLS

In [141]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [142]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

3491
971


In [143]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.274
Method:                 Least Squares   F-statistic:                     9.127
Date:                Sat, 28 Aug 2021   Prob (F-statistic):           2.40e-48
Time:                        22:03:22   Log-Likelihood:                -1808.8
No. Observations:                 971   AIC:                             3710.
Df Residuals:                     925   BIC:                             3934.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2903      1.379      0.211      0.833      -2.416       2.996
is_changeTeam        -0.1935      0.120     -1.615      0.107      -0.429       0.042
Age                  -0.1031      0.020     -5.093      0.000      -0.143      -0.063
Age^2              5.018e-05   9.88e-06      5.081      0.000    3.08e-05    6.96e-05
YRS_left              0.1737      0.035      5.030      0.000       0.106       0.241
AAV_dollar            0.2954      0.078      3.794      0.000       0.143       0.448
is_1year_contract     0.0093      0.163      0.057      0.954      -0.310       0.329
is_END_year          -0.8902      0.153     -5.821      0.000      -1.190      -0.590
Team_ATL             -0.7184      0.362     -1.985      0.047      -1.429      -0.008
Team_BAL             -0.9766      0.438     -2.231      0.026      -1.836      -0.118
Team_BOS             -0.4281      0.355     -1.205      0.229      -1.125       0.269
Team_CHC             -0.6797      0.388     -1.753      0.080      -1.441       0.081
Team_CHW             -0.6262      0.406     -1.541      0.124      -1.424       0.171
Team_CIN             -0.4769      0.391     -1.219      0.223      -1.245       0.291
Team_CLE             -0.6872      0.361     -1.903      0.057      -1.396       0.022
Team_COL             -1.4219      0.414     -3.437      0.001      -2.234      -0.610
Team_DET             -0.9888      0.410     -2.415      0.016      -1.793      -0.185
Team_HOU              0.3342      0.395      0.846      0.398      -0.441       1.110
Team_KC              -0.6872      0.415     -1.654      0.098      -1.503       0.128
Team_LAA             -0.4486      0.363     -1.236      0.217      -1.161       0.264
Team_LAD              0.3594      0.408      0.881      0.378      -0.441       1.160
Team_MIA             -0.7438      0.397     -1.871      0.062      -1.524       0.036
Team_MIL              0.2403      0.378      0.636      0.525      -0.501       0.982
Team_MIN             -0.5384      0.437     -1.233      0.218      -1.395       0.318
Team_NYM             -0.6074      0.378     -1.607      0.108      -1.349       0.134
Team_NYY             -0.1602      0.368     -0.435      0.663      -0.882       0.562
Team_OAK             -0.1194      0.425     -0.281      0.779      -0.954       0.715
Team_PHI             -0.7032      0.415     -1.693      0.091      -1.519       0.112
Team_PIT             -0.2873      0.444     -0.647      0.518      -1.159       0.584
Team_SD              -1.0928      0.416     -2.624      0.009      -1.910      -0.275
Team_SEA              0.3332      0.416      0.800      0.424      -0.484       1.150
Team_SF              -0.8841      0.396     -2.231      0.026      -1.662      -0.106
Team_STL             -0.0734      0.378     -0.194      0.846      -0.816

##  FE

In [145]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [146]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID"
]

ID_cols = ["Player_ID", "Season"]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

3491
971


In [147]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3064
Estimator:                   PanelOLS   R-squared (Between):          -1.444e+05
No. Observations:                 971   R-squared (Within):               0.3064
Date:                Sat, Aug 28 2021   R-squared (Overall):           -5.67e+04
Time:                        22:03:37   Log-likelihood                   -1470.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      7.2645
Entities:                         343   P-value                           0.0000
Avg Obs:                       2.8309   Distribution:                  F(36,592)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             10.569
                            

# Model 2

- WAR = (複數年)合約剩兩年       + 一年合約dummy + 控制變數
    - (複數年)合約剩三年
    - (複數年)合約剩四年以上
    - 基準: (複數年)合約剩一年

## OLS

In [148]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [149]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
 "Player_ID"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

3491
971


In [150]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     8.584
Date:                Sat, 28 Aug 2021   Prob (F-statistic):           2.90e-47
Time:                        22:04:10   Log-Likelihood:                -1807.8
No. Observations:                 971   AIC:                             3714.
Df Residuals:                     922   BIC:                             3953.
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.2726      1.390      0.196      0.845      -2.455       3.000
is_changeTeam        -0.1833      0.121     -1.519      0.129      -0.420       0.054
Age                  -0.1036      0.020     -5.103      0.000      -0.143      -0.064
Age^2              5.042e-05    9.9e-06      5.091      0.000     3.1e-05    6.99e-05
YRS_left              0.2357      0.061      3.880      0.000       0.116       0.355
AAV_dollar            0.2982      0.079      3.795      0.000       0.144       0.452
is_1year_contract    -0.0672      0.189     -0.356      0.722      -0.437       0.303
is_END_year          -0.9016      0.154     -5.846      0.000      -1.204      -0.599
is_left_2years       -0.1696      0.185     -0.918      0.359      -0.532       0.193
is_left_3years       -0.1427      0.246     -0.581      0.561      -0.625       0.339
is_left_ge4years     -0.4282      0.336     -1.276      0.202      -1.087       0.230
Team_ATL             -0.7074      0.362     -1.953      0.051      -1.418       0.004
Team_BAL             -0.9717      0.438     -2.218      0.027      -1.832      -0.112
Team_BOS             -0.4201      0.356     -1.181      0.238      -1.118       0.278
Team_CHC             -0.6649      0.388     -1.713      0.087      -1.427       0.097
Team_CHW             -0.6195      0.407     -1.522      0.128      -1.418       0.179
Team_CIN             -0.4736      0.392     -1.210      0.227      -1.242       0.295
Team_CLE             -0.6699      0.362     -1.852      0.064      -1.380       0.040
Team_COL             -1.4320      0.414     -3.458      0.001      -2.245      -0.619
Team_DET             -0.9931      0.410     -2.421      0.016      -1.798      -0.188
Team_HOU              0.3522      0.396      0.890      0.374      -0.424       1.129
Team_KC              -0.6585      0.416     -1.582      0.114      -1.475       0.159
Team_LAA             -0.4548      0.363     -1.251      0.211      -1.168       0.259
Team_LAD              0.3444      0.409      0.843      0.399      -0.457       1.146
Team_MIA             -0.7410      0.398     -1.862      0.063      -1.522       0.040
Team_MIL              0.2515      0.378      0.665      0.506      -0.491       0.994
Team_MIN             -0.5451      0.437     -1.248      0.212      -1.403       0.312
Team_NYM             -0.5886      0.379     -1.555      0.120      -1.331       0.154
Team_NYY             -0.1467      0.369     -0.398      0.691      -0.870       0.577
Team_OAK             -0.1336      0.426     -0.314      0.754      -0.969       0.702
Team_PHI             -0.7490      0.417     -1.795      0.073      -1.568       0.070
Team_PIT             -0.2985      0.444     -0.672      0.502      -1.171       0.574
Team_SD              -1.1251      0.418     -2.694      0.007      -1.945

##  FE

In [151]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [152]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
#  'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
 'is_FIRST_year_BigContract',
#  'is_END_year',
#  'is_left_2years',
#  'is_left_3years',
#  'is_left_ge4years',
 "type",
#  "Player_ID"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

3491
971


In [153]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3120
Estimator:                   PanelOLS   R-squared (Between):          -1.496e+05
No. Observations:                 971   R-squared (Within):               0.3120
Date:                Sat, Aug 28 2021   R-squared (Overall):          -5.877e+04
Time:                        22:04:54   Log-likelihood                   -1466.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.8494
Entities:                         343   P-value                           0.0000
Avg Obs:                       2.8309   Distribution:                  F(39,589)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             10.683
                            

#  Model 3

WAR = 複數年合約的第一年  + 一年合約dummy + 控制變數

## OLS

In [154]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [155]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
 'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
 "Player_ID"
]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic["WAR"].copy(deep=True)

3491
971


In [156]:
# OLS
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     8.353
Date:                Sat, 28 Aug 2021   Prob (F-statistic):           1.30e-43
Time:                        22:07:57   Log-Likelihood:                -1821.6
No. Observations:                 971   AIC:                             3735.
Df Residuals:                     925   BIC:                             3960.
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.1619      1.401     -0.116      0.908      -2.912       2.588
is_changeTeam                -0.2430      0.126     -1.924      0.055      -0.491       0.005
Age                          -0.1360      0.019     -7.002      0.000      -0.174      -0.098
Age^2                      6.623e-05   9.47e-06      6.990      0.000    4.76e-05    8.48e-05
AAV_dollar                    0.4213      0.075      5.643      0.000       0.275       0.568
is_1year_contract            -0.1059      0.168     -0.632      0.528      -0.435       0.223
is_FIRST_year_BigContract    -0.1051      0.146     -0.721      0.471      -0.391       0.181
is_END_year                  -0.9669      0.154     -6.271      0.000      -1.270      -0.664
Team_ATL                     -0.7407      0.367     -2.019      0.044      -1.460      -0.021
Team_BAL                     -1.0250      0.444     -2.310      0.021      -1.896      -0.154
Team_BOS                     -0.4723      0.360     -1.312      0.190      -1.179       0.234
Team_CHC                     -0.5842      0.393     -1.488      0.137      -1.355       0.186
Team_CHW                     -0.6739      0.412     -1.637      0.102      -1.482       0.134
Team_CIN                     -0.2863      0.395     -0.726      0.468      -1.060       0.488
Team_CLE                     -0.7134      0.366     -1.949      0.052      -1.432       0.005
Team_COL                     -1.4897      0.419     -3.554      0.000      -2.312      -0.667
Team_DET                     -0.8888      0.414     -2.144      0.032      -1.702      -0.075
Team_HOU                      0.3213      0.401      0.802      0.423      -0.465       1.108
Team_KC                      -0.6761      0.421     -1.605      0.109      -1.503       0.151
Team_LAA                     -0.2824      0.366     -0.771      0.441      -1.001       0.437
Team_LAD                      0.4874      0.413      1.181      0.238      -0.322       1.297
Team_MIA                     -0.7191      0.403     -1.785      0.075      -1.510       0.071
Team_MIL                      0.1716      0.383      0.448      0.654      -0.580       0.923
Team_MIN                     -0.6023      0.442     -1.362      0.173      -1.470       0.265
Team_NYM                     -0.5982      0.383     -1.561      0.119      -1.350       0.154
Team_NYY                     -0.1743      0.373     -0.467      0.640      -0.906       0.558
Team_OAK                     -0.2102      0.430     -0.488      0.626      -1.055       0.635
Team_PHI                     -0.6312      0.421     -1.500      0.134      -1.457       0.195
Team_PIT                     -0.3992      0.449     -0.889      0.374      -1.281       0.482
Team_SD                      -0.8787      0.420     -2.093      0.037      

##  FE

In [158]:
# df_pic = df_pitch.copy(deep=True) # 投手
df_pic = df_bat.copy(deep=True) # 野手

In [159]:
X_cols = df_pic.columns.tolist()
remove_lst = [
 '_idx',
#  'Season',
 'Player',
 'Team',
 'WAR',
#  'is_changeTeam',
#  'Age',
#  'Age^2',
 'Pos',
 'YRS',
 'YRS_left',
#  'AAV_dollar',
#  'is_1year_contract',
#  'is_FIRST_year_BigContract',
#  'is_END_year',
 'is_left_2years',
 'is_left_3years',
 'is_left_ge4years',
 "type",
#  "Player_ID"
]

ID_cols = ["Player_ID", 
           "Season"
          ]

for col in remove_lst:
    X_cols.remove(col)

# dropna
print(len(df_pic))
df_pic = df_pic[X_cols+["WAR"]].dropna()
print(len(df_pic))

X = df_pic[X_cols].copy(deep=True)
y = df_pic[["WAR"]+ID_cols].copy(deep=True)

3491
971


In [160]:
# FE
X = X.set_index(ID_cols)
X = sm.add_constant(X)

y = y.set_index(ID_cols)

fe = PanelOLS(y, X, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = fe.fit(cov_type='clustered', cluster_entity=True)
print(res)

C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\EMCT_Jeff\anaconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Pos_2B, Pos_3B, Pos_C, Pos_CF, Pos_DH, Pos_LF, Pos_OF, Pos_RF, Pos_SS

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.2965
Estimator:                   PanelOLS   R-squared (Between):          -1.133e+05
No. Observations:                 971   R-squared (Within):               0.2965
Date:                Sat, Aug 28 2021   R-squared (Overall):          -4.449e+04
Time:                        22:08:38   Log-likelihood                   -1477.8
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.9300
Entities:                         343   P-value                           0.0000
Avg Obs:                       2.8309   Distribution:                  F(36,592)
Min Obs:                       1.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             11.047
                            